<a href="https://colab.research.google.com/github/erickmu1/Image-Segmentation/blob/E/ImageSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# APS360: Image Segmentation #

### **Team 5** ###
- Bonnie He
- Erick Mejia Uzeda
- Hannah Lee

## Project Description ##

TODO

# Imports + Global Variables #

Here we import all required libraries and define any useful variables. (Feel free to improve this description)

In [34]:
# Pytorch
import torch
import torch.nn as nn

# Math
import numpy as np
import matplotlib.pyplot as plt

# Data storage/loading
import os
import pickle

# Global Variables
ROOT = '\\'
BACKGROUND_ID = 0 # TODO: actually determine a unique ID for 'background'

# Data Loading #

In [ ]:
# Code for loading data

# Data Pre-processing #

## Pre-computing Features from other ML models ##

We may have parts of the full ML pipeline implemented and others not. To speed up the training process, it is beneficial to precompute features resulting from a model and save them.

**Note:** Features are saved in such a format that once re-loaded, they can be passed directly into a `DataLoader`.

In [65]:
# Compute and Save Features from 'model'
def save_features(model, data_loader, file_name, dir=ROOT, use_cuda=False):
  features = []

  for input, label in data_loader:
    # Enable CUDA
    if use_cuda and torch.cuda.is_available():
      input = input.cuda()
      label = label.cuda()
    
    # Compute features
    with torch.no_grad():
      output = model(input)

    # Cache resulting features
    features.extend(output.cpu())
  
  # Save computed features using pickle
  save_path = os.path.join(dir, file_name + '.pickle')

  with open(save_path, 'wb+') as f:
    pickle.dump(features, f)

In [ ]:
# Load features
def load_features(file_name, dir=ROOT):
  file_path = os.join.path(dir, file_name + '.pickle')

  # Load features using pickle
  with open(file_path, 'rb') as f:
    features = pickle.load(f)
  
  return features

## Grouping Background Segments ##

Our dataset has many labels for different categories. Since our goal is to segment non-background items that are *distinct*, we will pre-process the raw segmentation maps to group relevant labels/categories that could be considered as *background*.

The following are grouped together: (Background)
- `floor`
- `wall`
- `ceiling`
- `window`

In [40]:
# Group all 'background' segments into one category
def group_background(seg_maps):
  # ASSUME: seg_maps is a np.array with dimensions (num_samples x [image_dims])
  # NOTE: this function modifies seg_maps itself!

  relevant_labels = [ 'floor', 'wall', 'ceiling', 'window' ]
  relevant_ids = [ 0, 1, 2, 3 ]  # TODO: get *actual* numerical IDs

  # Retrieve indices (pixels) of relevant labels
  indices = np.isin(seg_maps, relevant_ids)
  
  # Associate such indices (pixels) as 'background'
  seg_maps[indices] = BACKGROUND_ID

## Dealing with Label Permutation and Number of Segments ##

Our model will not particularly care what label is associated to segments, hence we prefer to refer to a segment by its segment ID. To re-iterate, our model does not care about what the value of the segment ID is for any given segment, what will dictate if our model is working if it can properly distinguish two **distinct** segments. Thus given an outputted segmentation map, any same segmentation map but with the segment IDs permutted is equally valid. Moreover, our model (in theory) should be invariant to more than permutation, that is any value can be assigned to a segment, as long as distinct segments have distinct IDs.

Next, the number of segments depends generally on the number of objects in the scene, so it not known a priori for new samples. This suggests the use of a recurrent architecture but once again, the order in which the segments would be generated is not of importance. In general, dealing with a variable number of possible segments must be accounted for in the model!

Ideas:
- Use a permutation invariant loss function:https://openreview.net/pdf?id=rJxpuoCqtQ
  - A major issue is that the number of segments produced for an image is not fixed, whereas this paper assumes the number of features `F` is known.
  - Should we not upper bound the outputs and round each pixel value to the nearest integer as to define its *segment ID*? How sensitive to small weight variantions would our model be? How dependent on the segment ID values temselves will the model be?
- Should we use a Recurrent Architecture?
  - That is: for each image
    1. extract the "largest" label
    2. remove all pixels from input that correspond to the segment meant to be removed
    3. feed in new image and repeat from step 1.
  - Note that we will need to either pre-process many images during training (which will make it slow) or cache multiple variants of an image which remove one segment at a time (can grow large if image has many segments)
    - Also need to pre-process and "order" segments by how "big" they are (need to compute pixel area of each segment and order them before making "inputs" that have the relevant segments removed)
  - We could possibly apply a *Recurrent Pipeline* to our baseline model (k-means) too!
- Say we generate all segments from the image in one-pass of the model (simple autoencoder architecture)
  - How do we deal with segment ID permutations (and even just different IDs in general, as long as the correct distinct segments have distinct IDs)?
  - How do we encode each segment ID?

**KEY IDEA:** What if we followed the convention that the largest segment (other than the background) must have the largest (smallest?) ID? Could this additional rule be learned by the model?
- If yes, then we resolved the issue of segment ID permutation
  - What happens for segments of comparable size? (to think about later)
- Could we then also apply *integer thresholding* (or some other thresholding) to identify each distinct segment? This would avoid the need to use a recurrent architecture

In [ ]:
# Code (if any) for pre-processing dataset to account for segment ID permutation

## Data Augmentation (Optional) ##

See if we can apply homographies on RGBD + ground truth segmentation map and if we can add noise to the RGBD images.

In [ ]:
# Code for homographic transformation

In [ ]:
# Code for adding noise to {RGB, Depth}

# Baseline model: k-means Algorithm #

In [ ]:
# Code for k-means algorithm

# Training Pipeline #

In [ ]:
# Code for the training pipeline

# Model Implementation #

In [ ]:
# Code for implementing the propsed model